# Image Comparison with Image Quality Assessment (IQA)

This notebook can be used to compare two images / volumes using different IQA metrics. The metrics used are:

| Metric    | Name                                          | Type | Dimensional behaviour | Colour Behaviour | Range              | Tested   | Validated |
|-----------|-----------------------------------------------|------|-----------------------|------------------|--------------------|----------|-----------|
| PSNR      | Peak Signal to Noise Ratio                    | FR   | 3D native             | OK               | $[0, \infty)$      | OK       | OK        |
| RMSE      | Root Mean Square Error                        | FR   | 3D native             | OK               | $(\infty, 0]$      | OK       | OK        |
| SSIM      | Structured Similarity                         | FR   | 3D native             | (OK) [^a]        | $[-1, 1]$ [^b]     | OK       | OK        |
| MS-SSIM   | Multi-Scale Structural Similarity             | FR   | 3D slicing            | ?                | $[0, 1]$           | x        | OK        |
| FSIM      | Feature Similarity Index                      | FR   | 3D slicing            | OK               | $[0, 1]$           | OK       | OK        |
| VIFp      | Visual Information Fidelity in *pixel* domain | FR   | 3D slicing            | ?                | $[0, \infty)$ [^c] | x        | x         |
| VSI       | Visual Saliency Index                         | FR   | 3D slicing            | OK [^d]          | $[0, 1]$           | x        | x         |
| MAD       | Most Apparent Distortion                      | FR   | 3D slicing            |                  | $[0, \infty)$      | OK       | x         |
| GSM       | Gradient Similarity                           | FR   | 3D native or slicing  |                  | $[0, 1]$           | x        | x         |
| CNR       | Contrast to Noise Ratio                       | NR   | 3D native             |                  | $[0, \infty)$      | OK       | x         |
| SNR       | Signal to Noise Ratio                         | NR   | 3D native             | OK               | $[0, \infty)$      | OK       | x         |
| Q-Measure | Q-Measure                                     | NR   | 3D only [^e]          | x                | $[0, \infty)$      | x        | x         |

**Important Notes:**
The calculated values for VIFp are probably not correct in this implementation. Those values should be treated with caution. Further testing is required.
The MAD metric takes very long for calculation. It is recommended to use it only for small slices of the images.
The largest images currently tested are around `2000x2300x2300` pixels. Calculation for these works with 512GiB of RAM.

**Usage:**
The metrics are implemented to calculate the scores for an 16-bit data range (0-65535) per default in this notebook. The data range can be changed by setting the `data_range` variable for each metric. Images are first loaded from .raw files or .mhd files and their corresponding .raw file, normalized to the chosen data range and then compared. The scores are then calculated and printed. The images are plotted to visually compare them. File names need to be given with the bit depth denoted as a suffix (e.g. `_8bit.raw`, `_16bit.raw`) and the dimensions of the images need to be given in the file name (e.g. `_512x512x512_`). The images are assumed to be grayscale.

To use this notebook, first set the filepaths and parameters, then press 'Run All'.

**Requirements:**
The following packages have to be installed:
- jupyter
- matplotlib
- nibabel
- numpy
- piq
- pytorch
- scikit-image
- scipy

**References:**
-  Wang, Z., Bovik, A. C., Sheikh, H. R., & Simoncelli, E. P. (2004). Image quality 
assessment: From error visibility to structural similarity. IEEE Transactions on 
Image Processing, 13(4), 600–612. <https://doi.org/10.1109/TIP.2003.819861>
- Wang, Z., Simoncelli, E. P., & Bovik, A. C. (2003). Multi-scale structural 
similarity for image quality assessment. The Thirty-Seventh Asilomar Conference on 
Signals, Systems & Computers, 1298–1402. <https://doi.org/10.1109/ACSSC.2003.1292216>
- Zhang, L., Zhang, L., Mou, X., & Zhang, D. (2011). FSIM: A feature similarity 
index for image quality assessment. IEEE Transactions on Image Processing, 20(8). 
<https://doi.org/10.1109/TIP.2011.2109730>
- Sheikh, H. R., & Bovik, A. C. (2006). Image information and visual quality. IEEE 
Transactions on Image Processing, 15(2), 430–444. 
<https://doi.org/10.1109/TIP.2005.859378>
- Zhang, L., Shen, Y., & Li, H. (2014). VSI: A visual saliency-induced index for 
perceptual image quality assessment. IEEE Transactions on Image Processing, 23(10), 
4270–4281. <https://doi.org/10.1109/TIP.2014.2346028>
- Larson, E. C., & Chandler, D. M. (2010). Most apparent distortion: full-reference 
image quality assessment and the role of strategy. Journal of Electronic Imaging, 19
(1), 011006. <https://doi.org/10.1117/1.3267105>
- Liu, A., Lin, W., & Narwaria, M. (2012). Image quality assessment based on 
gradient similarity. IEEE Transactions on Image Processing, 21(4), 1500–1512. 
<https://doi.org/10.1109/TIP.2011.2175935>
- Desai, N., Singh, A., & Valentino, D. J. (2010). Practical evaluation of image 
quality in computed radiographic (CR) imaging systems. Medical Imaging 2010: Physics 
of Medical Imaging, 7622, 76224Q. <https://doi.org/10.1117/12.844640>
- Reiter, M., Weiß, D., Gusenbauer, C., Erler, M., Kuhn, C., Kasperl, S., & 
Kastner, J. (2014). Evaluation of a Histogram-based Image Quality Measure for X-ray 
computed Tomography. 5th Conference on Industrial Computed Tomography (iCT) 2014, 25-28 
February 2014, Wels, Austria. e-Journal of Nondestructive Testing Vol. 19(6). 
https://www.ndt.net/?id=15715

[^a]: The metric is calculated channel-wise for color images. The values are then averaged after weighting.
[^b]: The range for SSIM is given as $[-1, 1]$, but is usually $[0, 1]$ in practice.
[^c]: Normally $[0, 1]$, but can be higher than 1 for modified images with higher 
contrast than reference images.
[^d]: The original metric supports RGB images only. This implementation can work 
with grayscale images by copying the luminance channel 3 times.
[^e]: The Q-Measure is a special metric designed for CT images. Therefore it only works
with 3D volumes.

## Import

In [ ]:
import os

import skimage.transform as skt
import viqa
from viqa import MultipleMetrics, load_data

## Data Loading and Setup

In [ ]:
# Change these filepaths to your images (_r = reference, _m = modified)
file_path_img_r = "/path/to/reference/image"
file_name_img_r = "reference_image_512x512x512_16bit.raw"
file_path_img_m = "/path/to/modified/image"
file_name_img_m = "modified_image_512x512x512_16bit.raw"

# Set these to true if you want to calculate the denominated metric for your data
calc_psnr = True
calc_rmse = True
calc_ssim = True
calc_msssim = False  # Not possible for very large images due to memory constraints
calc_fsim = True
calc_vif = False  # Not possible for very large images due to memory constraints
calc_vsi = False  # Not possible for very large images due to memory constraints
calc_mad = False  # Not possible for very large images due to memory constraints
calc_gsm = True

calc_cnr = True
calc_snr = True
calc_qmeasure = True

In [ ]:
# Data loading
file_path_img_r = os.path.join(file_path_img_r, file_name_img_r)
file_path_img_m = os.path.join(file_path_img_m, file_name_img_m)
img_r = load_data(file_path_img_r, data_range=65535, normalize=False, batch=False)
img_m = load_data(file_path_img_m, data_range=65535, normalize=False, batch=False)

metrics = []
metrics_parameters = []

In [ ]:
# Print shapes
print("Original image shape:", img_r.shape)
print("Modified shape:", img_m.shape)

In [ ]:
# Resize image if shapes unequal
if img_r.shape != img_m.shape:
    img_m = skt.resize(img_m, img_r.shape, preserve_range=True, order=1)
    img_m = img_m.astype(img_r.dtype)

## Calculation

### CNR

Set the parameters for CNR.

In [ ]:
print("Shape of modified image:", img_m.shape)

In [ ]:
# Change these parameters
cnr_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
cnr_calc_parameters = {
    "background_center": (50, 50, 50),
    "signal_center": (200, 200, 200),
    "radius": 10,
}

In [ ]:
# Run this to visualize centers
if calc_cnr:
    metric_cnr = viqa.CNR(**cnr_load_parameters)
    metric_cnr.visualize_centers(img=img_m, signal_center=cnr_calc_parameters["signal_center"], background_center=cnr_calc_parameters["background_center"], radius=cnr_calc_parameters["radius"])

### SNR

Set the parameters for SNR.

In [ ]:
# Change these parameters
snr_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
snr_calc_parameters = {
    "signal_center": (200, 200, 200),
    "radius": 10,
}

In [ ]:
# Run this to visualize center
if calc_snr:
    metric_snr = viqa.SNR(**snr_load_parameters)
    metric_snr.visualize_centers(img=img_m, signal_center=snr_calc_parameters["signal_center"], radius=snr_calc_parameters["radius"])

### PSNR
Set the parameters for PSNR.

In [ ]:
# Change these parameters
psnr_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
psnr_calc_parameters = {}

### RMSE
Set the parameters for RMSE.

In [ ]:
# Change these parameters
rmse_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
rmse_calc_parameters = {}

### SSIM
Set the parameters for SSIM. Alpha, beta and gamma control the influence of the luminance, contrast and structure terms respectively. The final SSIM score is lower if one of these parameters is set to a higher value due to $SSIM = luminance ^ \alpha \times contrast ^ \beta \times structure ^ \gamma$.

In [ ]:
# Change these parameters
ssim_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
ssim_calc_parameters = {
    "gaussian_weights": True,
    "use_sample_covariance": False,
    "sigma": 1.5,
    "alpha": 1,
    "beta": 1,
    "gamma": 1,
}

### MS-SSIM

Set the parameters for MS-SSIM.

In [ ]:
# Change these parameters
msssim_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
msssim_calc_parameters = {
    "dim": 0,
}

### FSIM
Set the parameters for FSIM. The chromatic parameter can be set to true if the images are RGB images.

In [ ]:
# Change these parameters
fsim_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
fsim_calc_parameters = {
    "dim": 0,
}

### VIFp
Set the parameters for VIFp. The sigma_n_sq parameter can be set to adapt the metric for the variance of visual noise. This parameter is set to 3.2 per default and has to be changed according to the data. Currently, there is no recommendation how to set this parameter.

In [ ]:
# Change these parameters
vif_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
vif_calc_parameters = {
    "dim": 0,
    "sigma_n_sq": 2,
}

### VSI

Set the parameters for VSI.

In [ ]:
# Change these parameters
vsi_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
vsi_calc_parameters = {
    "dim": 0,
}

### MAD

Set the parameters for MAD. 

In [ ]:
# Change these parameters
mad_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
mad_calc_parameters = {
    "dim": 0,
    "block_size": 16,
    "block_overlap": 0.75,
    "beta_1": 0.467,
    "beta_2": 0.130,
    "luminance_function": {"b": 0, "k": 0.02874, "gamma": 2.2},
    "orientations_num": 4,
    "scales_num": 5,
    "weights": [0.5, 0.75, 1, 5, 6],
}

### GSM

Set the parameters for GSM.

In [ ]:
# Change these parameters
gsm_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
gsm_calc_parameters = {
    "dim": 0,
    "experimental": False,
    "c": 200,
    "p": 0.1,
}

### Q-Measure

Set the parameters for QMeasure.

In [ ]:
# Change these parameters
qmeasure_load_parameters = {
    "data_range": 65535,
    "normalize": False,
    "chromatic": False,
}
qmeasure_calc_parameters = {
    "hist_bins": 128,
    "num_peaks": 2,
}

## Calculation

In [ ]:
if calc_psnr:
    metric_psnr = viqa.PSNR(**psnr_load_parameters)
    metrics.append(metric_psnr)
    metrics_parameters.append(psnr_calc_parameters)
if calc_rmse:
    metric_rmse = viqa.RMSE(**rmse_load_parameters)
    metrics.append(metric_rmse)
    metrics_parameters.append(rmse_calc_parameters)
if calc_ssim:
    metric_ssim = viqa.SSIM(**ssim_load_parameters)
    metrics.append(metric_ssim)
    metrics_parameters.append(ssim_calc_parameters)
if calc_msssim:
    metric_msssim = viqa.MSSSIM(**msssim_load_parameters)
    metrics.append(metric_msssim)
    metrics_parameters.append(msssim_calc_parameters)
if calc_fsim:
    metric_fsim = viqa.FSIM(**fsim_load_parameters)
    metrics.append(metric_fsim)
    metrics_parameters.append(fsim_calc_parameters)
if calc_vif:
    metric_vif = viqa.VIFp(**vif_load_parameters)
    metrics.append(metric_vif)
    metrics_parameters.append(vif_calc_parameters)
if calc_vsi:
    metric_vsi = viqa.VSI(**vsi_load_parameters)
    metrics.append(metric_vsi)
    metrics_parameters.append(vsi_calc_parameters)
if calc_mad:
    metric_mad = viqa.MAD(**mad_load_parameters)
    metrics.append(metric_mad)
    metrics_parameters.append(mad_calc_parameters)
if calc_gsm:
    metric_gsm = viqa.GSM(**gsm_load_parameters)
    metrics.append(metric_gsm)
    metrics_parameters.append(gsm_calc_parameters)
if calc_cnr:
    metrics.append(metric_cnr)
    metrics_parameters.append(cnr_calc_parameters)
if calc_snr:
    metrics.append(metric_snr)
    metrics_parameters.append(snr_calc_parameters)
if calc_qmeasure:
    metric_qmeasure = viqa.QMeasure(**qmeasure_load_parameters)
    metrics.append(metric_qmeasure)
    metrics_parameters.append(qmeasure_calc_parameters)

In [ ]:
# Calculate metrics
multiple = MultipleMetrics(metrics, metrics_parameters)
results = multiple.calculate(img_r, img_m)

## Report

In [ ]:
# Change types of reports you want to generate
# img_r, img_m, x (alternatively y or z) are for parameter ``image``.
multiple.report(csv=True, metadata=True, text=True, image=True, file_path="./results", export_image=True, img_r=img_r, img_m=img_m, x=200)